In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import os
from os import listdir
from os.path import isdir, join
from PIL import Image
import matplotlib.pyplot as plt

# Configuration
class Config:
    DATASET_PATH = "UCSD_Anomaly_Dataset.v1p2/UCSDped1/Train"
    SINGLE_TEST_PATH = "UCSD_Anomaly_Dataset.v1p2/UCSDped1/Test/Test032"
    TEST_DATA_PATH = "UCSD_Anomaly_Dataset.v1p2/UCSDped1/Test"
    BATCH_SIZE = 2
    EPOCHS = 3
    MODEL_PATH = "model_lstm.pth"
    EARLY_STOPPING_PATIENCE = 5  # Stop if validation loss doesn't improve for 5 epochs


In [2]:
def simple_progress_bar(iteration, total, bar_length=50):
    progress = (iteration / total)
    arrow = '-' * int(round(progress * bar_length) - 1) + '>'
    spaces = ' ' * (bar_length - len(arrow))

    print('\r[{}] {}/{} ({:.2f}%)'.format(arrow + spaces, iteration, total, progress * 100), end='')


In [3]:
from os import listdir
from os.path import isfile, join, isdir
from PIL import Image
import numpy as np
import shelve
def get_clips_by_stride(stride, frames_list, sequence_size):
    """ For data augmenting purposes.
    Parameters
    ----------
    stride : int
        The desired distance between two consecutive frames
    frames_list : list
        A list of sorted frames of shape 128 X 128
    sequence_size: int
        The size of the desired LSTM sequence
    Returns
    -------
    list
        A list of clips, 10 frames each
    """
    clips = []
    sz = len(frames_list)
    clip = np.zeros(shape=(sequence_size, 128, 128, 1))
    cnt = 0
    for start in range(0, stride):
        for i in range(start, sz, stride):
            clip[cnt, :, :, 0] = frames_list[i]
            cnt = cnt + 1
            if cnt == sequence_size:
                clips.append(np.copy(clip))
                cnt = 0
    return clips



def get_training_set():
    clips = []
    for f in sorted(listdir(Config.DATASET_PATH)):
        if isdir(join(Config.DATASET_PATH, f)):
            all_frames = []
            for c in sorted(listdir(join(Config.DATASET_PATH, f))):
                if str(join(join(Config.DATASET_PATH, f), c))[-3:] == "tif":
                    img = Image.open(join(join(Config.DATASET_PATH, f), c)).resize((128, 128))  # Reduced image size
                    img = np.array(img, dtype=np.float32) / 128.0
                    all_frames.append(img)
            for stride in range(1, 3):
                clips.extend(get_clips_by_stride(stride=stride, frames_list=all_frames, sequence_size=10))
    return np.array(clips)  # Convert list to numpy array before tensor conversion


In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torchvision.transforms import ToTensor




In [5]:
class ConvLSTMCell(nn.Module):
    def __init__(self, input_size, hidden_size, kernel_size):
        super(ConvLSTMCell, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.kernel_size = kernel_size
        self.padding = kernel_size // 2
        self.conv = nn.Conv2d(input_size + hidden_size, 4 * hidden_size, kernel_size, 1, self.padding)

    def forward(self, x, state):
        h_cur, c_cur = state
        combined = torch.cat([x, h_cur], dim=1)
        gates = self.conv(combined)
        ingate, forgetgate, cellgate, outgate = gates.chunk(4, 1)
        ingate = torch.sigmoid(ingate)
        forgetgate = torch.sigmoid(forgetgate)
        cellgate = torch.tanh(cellgate)
        outgate = torch.sigmoid(outgate)

        c_next = (forgetgate * c_cur) + (ingate * cellgate)
        h_next = outgate * torch.tanh(c_next)

        return h_next, c_next
    
    def initialize_hidden_state(self, batch_size, image_size):
        height, width = image_size
        return (torch.zeros(batch_size, self.hidden_size, height, width, device=self.conv.weight.device),
                torch.zeros(batch_size, self.hidden_size, height, width, device=self.conv.weight.device))



In [6]:
class ConvLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, kernel_size, num_layers):
        super(ConvLSTM, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.kernel_size = kernel_size
        self.num_layers = num_layers
        self._all_layers = []
        for i in range(0, self.num_layers):
            cur_input_dim = self.input_size if i == 0 else self.hidden_size
            cell = ConvLSTMCell(input_size=cur_input_dim,
                                hidden_size=self.hidden_size,
                                kernel_size=self.kernel_size)
            self._all_layers.append(cell)
            self.add_module('cell_{}'.format(i), cell)

    def forward(self, x, hidden_state=None):
        internal_state = []
        outputs = []
        for step in range(x.size(1)):
            x_t = x[:, step, :, :, :]
            for i in range(self.num_layers):
                # get or initialize the internal state
                if hidden_state is None:
                    h, c = self._all_layers[i].initialize_hidden_state(batch_size=x_t.size(0), image_size=(x_t.size(2), x_t.size(3)))
                else:
                    h, c = hidden_state[i]
                h, c = self._all_layers[i](x_t, [h, c])
                x_t = h
                if i == (self.num_layers - 1):
                    outputs.append(h)
            internal_state.append([h, c])

        layer_output = torch.stack(outputs, dim=1)
        return layer_output, internal_state



In [7]:
class AnomalyDetectionModel(nn.Module):
    def __init__(self):
        super(AnomalyDetectionModel, self).__init__()
        # Encoder
        self.encoder_time_distributed_conv1 = nn.Conv2d(1, 128, kernel_size=11, stride=4, padding=5)
        self.encoder_time_distributed_conv2 = nn.Conv2d(128, 64, kernel_size=5, stride=2, padding=2)
        self.encoder_conv_lstm1 = ConvLSTM(64, 64, kernel_size=3, num_layers=1)
        self.encoder_conv_lstm2 = ConvLSTM(64, 32, kernel_size=3, num_layers=1)
        self.encoder_conv_lstm3 = ConvLSTM(32, 64, kernel_size=3, num_layers=1)

        # Decoder
        self.decoder_time_distributed_conv_transpose1 = nn.ConvTranspose2d(64, 64, kernel_size=5, stride=2, padding=2, output_padding=1)
        self.decoder_time_distributed_conv_transpose2 = nn.ConvTranspose2d(64, 128, kernel_size=11, stride=4, padding=2, output_padding=1)
        self.decoder_time_distributed_conv3 = nn.Conv2d(128, 1, kernel_size=10, padding=3)



    def forward(self, x):
        # Reshape to handle time distributed operations
        b, t, h, w, c = x.size()
        x = x.view(b*t, c, h, w)
        
        # Encoder
        x = self.encoder_time_distributed_conv1(x)
        
        x = self.encoder_time_distributed_conv2(x)
        
        # Reshape for LSTM layers
        x = x.view(b, t, x.size(1), x.size(2), x.size(3))
        
        x, _ = self.encoder_conv_lstm1(x)
        
        x, _ = self.encoder_conv_lstm2(x)
        
        x, _ = self.encoder_conv_lstm3(x)

        # Reshape for decoder conv layers
        x = x.view(b*t, x.size(2), x.size(3), x.size(4))
        
        # Decoder
        x = self.decoder_time_distributed_conv_transpose1(x)
        
        x = self.decoder_time_distributed_conv_transpose2(x)
        
        x = self.decoder_time_distributed_conv3(x)

        x = x[:, :, :128, :128]

        # Reshape back to original shape
        x = x.view(b, t, h, w, c)
        
        return x


In [8]:
def train_model(model, train_loader, optimizer, criterion, device, epochs):
    model.train()
    best_loss = float('inf')
    stopping_counter = 0

    epochs = epochs
    
    for epoch in range(epochs):
        epoch_loss = 0.0
        for sequences in train_loader:
            print(f"Type of sequences: {type(sequences)}, Structure: {sequences}")
            sequences = sequences.to(device)
            optimizer.zero_grad()
            b, t, h, w, c = sequences.size()
            sequences_reshaped = sequences.view(b*t, c, h, w)
            outputs = model(sequences)
            loss = criterion(outputs.view(b*t, c, h, w), sequences_reshaped)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()

        avg_epoch_loss = epoch_loss / len(train_loader)
        print(f"Epoch [{epoch+1}/{Config.EPOCHS}], Loss: {avg_epoch_loss:.4f}")

        # Early stopping based on training loss
        if avg_epoch_loss < best_loss:
            best_loss = avg_epoch_loss
            torch.save(model.state_dict(), Config.MODEL_PATH)
            stopping_counter = 0
        else:
            stopping_counter += 1
            if stopping_counter == Config.EARLY_STOPPING_PATIENCE:
                print("Early stopping triggered.")
                break

def get_model(reload_model=True):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = AnomalyDetectionModel().to(device)
    optimizer = optim.Adam(model.parameters(), lr=1e-4)
    criterion = nn.MSELoss()
    epochs = Config.EPOCHS

    if reload_model:
        model.load_state_dict(torch.load(Config.MODEL_PATH))
        model.eval()
    else:
        training_set = get_training_set()
        training_set = torch.tensor(training_set).float()
        # DataLoader with num_workers
        train_loader = DataLoader(training_set, batch_size=Config.BATCH_SIZE, shuffle=True, num_workers=4)  # num_workers set to 4
        train_model(model, train_loader, optimizer, criterion, device, epochs)
        torch.save(model.state_dict(), Config.MODEL_PATH)

    return model


def get_single_test():
    sz = 200
    test = np.zeros(shape=(sz, 128, 128, 1))
    cnt = 0
    for f in sorted(listdir(Config.SINGLE_TEST_PATH)):
        if str(join(Config.SINGLE_TEST_PATH, f))[-3:] == "tif":
            img = Image.open(join(Config.SINGLE_TEST_PATH, f)).resize((128, 128))
            img = np.array(img, dtype=np.float32) / 128.0
            test[cnt, :, :, 0] = img
            cnt = cnt + 1
    return test

import matplotlib.pyplot as plt


# Update evaluate function to use PyTorch for model prediction
def evaluate_batchwise(model, sequences, batch_size):
    reconstructed_sequences_list = []
    for i in range(0, sequences.size(0), batch_size):
        with torch.no_grad():
            batch = sequences[i:i+batch_size]
            reconstructed_batch = model(batch)
            reconstructed_sequences_list.append(reconstructed_batch)
    return torch.cat(reconstructed_sequences_list, dim=0)

def evaluate():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = get_model(True).to(device)
    test = get_single_test()
    test = torch.tensor(np.array(test)).float().to(device)

    sz = test.size(0) - 10 + 1
    sequences = torch.zeros(sz, 10, 128, 128, 1).to(device)
    for i in range(sz):
        sequences[i] = test[i:i+10].to(device)

    reconstructed_sequences = evaluate_batchwise(model, sequences, batch_size=10)

    sequences_reconstruction_cost = torch.sqrt(torch.sum((sequences - reconstructed_sequences) ** 2, dim=[2,3,4]))

    sa = (sequences_reconstruction_cost - torch.min(sequences_reconstruction_cost)) / torch.max(sequences_reconstruction_cost)
    sr = 1.0 - sa

    plt.plot(sr.cpu().numpy())
    plt.ylabel('regularity score Sr(t)')
    plt.xlabel('frame t')
    plt.show()



In [9]:

# Call evaluate to test the converted code
# evaluate()

In [10]:
from collections import OrderedDict
from typing import List, Tuple

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import CIFAR10

import flwr as fl
from flwr.common import Metrics

DEVICE = torch.device("cuda")  # Try "cuda" to train on GPU
print(
    f"Training on {DEVICE} using PyTorch {torch.__version__} and Flower {fl.__version__}"
)

Training on cuda using PyTorch 2.1.0+cu118 and Flower 1.5.0


In [11]:
NUM_CLIENTS = 10
BATCH_SIZE = 2


In [12]:
def get_test_set():
    test_clips_with_gt = []
    test_clips = []
    gt_clips = []

    if not os.path.exists(Config.TEST_DATA_PATH):
        raise FileNotFoundError(f"Test data path {Config.TEST_DATA_PATH} does not exist")

    for f in sorted(listdir(Config.TEST_DATA_PATH)):
        dir_path = join(Config.TEST_DATA_PATH, f)
        if isdir(dir_path):
            all_frames = []
            all_gt_frames = []  # Initialize here

            # Load normal test sequences
            if not f.endswith('_gt'):
                for c in sorted(listdir(dir_path)):
                    file_path = join(dir_path, c)
                    if file_path.endswith(".tif"):
                        print(f"Attempting to open: {file_path}")  # Debug print
                        if not os.path.isfile(file_path):
                            raise FileNotFoundError(f"Expected file not found: {file_path}")
                        try:
                            img = Image.open(file_path).resize((128, 128))
                            img = np.array(img, dtype=np.float32) / 128.0
                            all_frames.append(img)
                        except IOError as e:
                            print(f"Error opening file {file_path}: {e}")  # Additional error handling

            # Load ground truth sequences
            else:
                for c in sorted(listdir(dir_path)):
                    file_path = join(dir_path, c)
                    if file_path.endswith(".bmp"):
                        print(f"Attempting to open GT file: {file_path}")  # Debug print
                        if not os.path.isfile(file_path):
                            raise FileNotFoundError(f"Expected file not found: {file_path}")
                        try:
                            img = Image.open(file_path).convert('L').resize((128, 128))
                            img = np.array(img, dtype=np.float32) / 255.0  # Normalizing GT images
                            all_gt_frames.append(img)
                        except IOError as e:
                            print(f"Error opening GT file {file_path}: {e}")  # Additional error handling

            # Add to respective lists
            if all_frames:
                for stride in range(1, 3):
                    test_clips.extend(get_clips_by_stride(stride=stride, frames_list=all_frames, sequence_size=10))
            if all_gt_frames:
                for stride in range(1, 3):
                    gt_clips.extend(get_clips_by_stride(stride=stride, frames_list=all_gt_frames, sequence_size=10))
    
    for test_clip, gt_clip in zip(test_clips, gt_clips):
        test_clips_with_gt.append((test_clip, gt_clip))

    return test_clips_with_gt



In [13]:
def create_dataloader():
    # Load and preprocess the dataset
    training_set = get_training_set()

    # Convert the dataset to a PyTorch Tensor
    training_set_tensor = torch.tensor(training_set).float()

    # Create a TensorDataset
    train_dataset = TensorDataset(training_set_tensor)

    # Create a DataLoader
    train_loader = DataLoader(train_dataset, batch_size=Config.BATCH_SIZE, shuffle=True, num_workers=4)

    return train_loader

In [14]:
import torch
from torch.utils.data import DataLoader, TensorDataset

def create_test_loader():
    test_clips_with_gt = get_test_set()

    # Create TensorDataset
    test_clips, gt_clips = zip(*test_clips_with_gt)
    test_dataset = TensorDataset(torch.tensor(np.array(test_clips)).float(), 
                                 torch.tensor(np.array(gt_clips)).float())

    # Create DataLoader
    test_loader = DataLoader(test_dataset, batch_size=Config.BATCH_SIZE, shuffle=False, num_workers=4)

    return test_loader

# Usage
# test_loader = create_test_loader()

In [15]:
def parse_annotations():
    TestVideoFile = [
        {"gt_frame": [60, 152]},
        {"gt_frame": [50, 175]},
        {"gt_frame": [91, 200]},
        {"gt_frame": [31, 168]},
        {"gt_frame": [5, 90, 140, 200]},
        {"gt_frame": [1, 100, 110, 200]},
        {"gt_frame": [1, 175]},
        {"gt_frame": [1, 94]},
        {"gt_frame": [1, 48]},
        {"gt_frame": [1, 140]},
        {"gt_frame": [70, 165]},
        {"gt_frame": [130, 200]},
        {"gt_frame": [1, 156]},
        {"gt_frame": [1, 200]},
        {"gt_frame": [138, 200]},
        {"gt_frame": [123, 200]},
        {"gt_frame": [1, 47]},
        {"gt_frame": [54, 120]},
        {"gt_frame": [64, 138]},
        {"gt_frame": [45, 175]},
        {"gt_frame": [31, 200]},
        {"gt_frame": [16, 107]},
        {"gt_frame": [8, 165]},
        {"gt_frame": [50, 171]},
        {"gt_frame": [40, 135]},
        {"gt_frame": [77, 144]},
        {"gt_frame": [10, 122]},
        {"gt_frame": [105, 200]},
        {"gt_frame": [1, 15, 45, 113]},
        {"gt_frame": [175, 200]},
        {"gt_frame": [1, 180]},
        {"gt_frame": [1, 52, 65, 115]},
        {"gt_frame": [5, 165]},
        {"gt_frame": [1, 121]},
        {"gt_frame": [86, 200]},
        {"gt_frame": [15, 108]},
    ]

    annotations = {}
    for i, video in enumerate(TestVideoFile):
        abnormal_frames = set()
        for start, end in zip(video["gt_frame"][::2], video["gt_frame"][1::2]):
            abnormal_frames.update(range(start, end + 1))
        annotations[i] = abnormal_frames
    return annotations

annotations = parse_annotations()


In [16]:
import torch
import torch.nn.functional as F

def calculate_metrics(outputs, labels):
    # Assuming outputs and labels are tensors of the same shape
    # Adjust the calculation based on your model's output format
    predicted = outputs.round()  # Assuming model outputs a probability that needs to be rounded to 0 or 1
    correct = (predicted == labels).float()  # Correct predictions
    accuracy = correct.sum() / correct.numel()
    
    # Precision, Recall, and F1 Score can be calculated here as well, depending on your requirement

    return accuracy  #, precision, recall, f1_score

def evaluate(model, test_loader, device):
    model.eval()  # Set the model to evaluation mode

    total_accuracy = 0
    total_loss = 0
    criterion = nn.MSELoss()  # or any other appropriate loss function

    with torch.no_grad():
        for sequences, labels in test_loader:
            sequences, labels = sequences.to(device), labels.to(device)

            outputs = model(sequences)
            loss = criterion(outputs, labels)
            total_loss += loss.item()

            accuracy = calculate_metrics(outputs, labels)
            total_accuracy += accuracy.item()

    avg_loss = total_loss / len(test_loader)
    avg_accuracy = total_accuracy / len(test_loader)

    print(f'Evaluation Loss: {avg_loss:.4f}, Accuracy: {avg_accuracy:.4f}')

    return avg_loss, avg_accuracy

# Usage example
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = get_model().to(device)
test_loader = create_test_loader()
evaluate(model, test_loader, device)


Attempting to open: UCSD_Anomaly_Dataset.v1p2/UCSDped1/Test\Test001\001.tif
Attempting to open: UCSD_Anomaly_Dataset.v1p2/UCSDped1/Test\Test001\002.tif
Attempting to open: UCSD_Anomaly_Dataset.v1p2/UCSDped1/Test\Test001\003.tif
Attempting to open: UCSD_Anomaly_Dataset.v1p2/UCSDped1/Test\Test001\004.tif
Attempting to open: UCSD_Anomaly_Dataset.v1p2/UCSDped1/Test\Test001\005.tif
Attempting to open: UCSD_Anomaly_Dataset.v1p2/UCSDped1/Test\Test001\006.tif
Attempting to open: UCSD_Anomaly_Dataset.v1p2/UCSDped1/Test\Test001\007.tif
Attempting to open: UCSD_Anomaly_Dataset.v1p2/UCSDped1/Test\Test001\008.tif
Attempting to open: UCSD_Anomaly_Dataset.v1p2/UCSDped1/Test\Test001\009.tif
Attempting to open: UCSD_Anomaly_Dataset.v1p2/UCSDped1/Test\Test001\010.tif
Attempting to open: UCSD_Anomaly_Dataset.v1p2/UCSDped1/Test\Test001\011.tif
Attempting to open: UCSD_Anomaly_Dataset.v1p2/UCSDped1/Test\Test001\012.tif
Attempting to open: UCSD_Anomaly_Dataset.v1p2/UCSDped1/Test\Test001\013.tif
Attempting t

(0.6519395890831947, 0.3182889476418495)

In [17]:
import numpy as np
import os
from os import listdir
from os.path import isdir, join
from PIL import Image
import torch
from torch.utils.data import TensorDataset, DataLoader

def split_dataset(data, num_splits=10):
    """ Split the dataset into num_splits parts. """
    split_size = len(data) // num_splits
    return [data[i*split_size:(i+1)*split_size] for i in range(num_splits)]

def get_training_set():
    clips = []
    for f in sorted(listdir(Config.DATASET_PATH)):
        if isdir(join(Config.DATASET_PATH, f)):
            all_frames = []
            for c in sorted(listdir(join(Config.DATASET_PATH, f))):
                if str(join(join(Config.DATASET_PATH, f), c))[-3:] == "tif":
                    img = Image.open(join(join(Config.DATASET_PATH, f), c)).resize((128, 128))  # Reduced image size
                    img = np.array(img, dtype=np.float32) / 128.0
                    all_frames.append(img)
            for stride in range(1, 3):
                clips.extend(get_clips_by_stride(stride=stride, frames_list=all_frames, sequence_size=10))
    return split_dataset(np.array(clips))  # Split dataset into 10 parts

def get_test_set():
    test_clips_with_gt = []
    test_clips = []
    gt_clips = []

    if not os.path.exists(Config.TEST_DATA_PATH):
        raise FileNotFoundError(f"Test data path {Config.TEST_DATA_PATH} does not exist")

    for f in sorted(listdir(Config.TEST_DATA_PATH)):
        dir_path = join(Config.TEST_DATA_PATH, f)
        if isdir(dir_path):
            all_frames = []
            all_gt_frames = []  # Initialize here

            # Load normal test sequences
            if not f.endswith('_gt'):
                for c in sorted(listdir(dir_path)):
                    file_path = join(dir_path, c)
                    if file_path.endswith(".tif"):
                        print(f"Attempting to open: {file_path}")  # Debug print
                        if not os.path.isfile(file_path):
                            raise FileNotFoundError(f"Expected file not found: {file_path}")
                        try:
                            img = Image.open(file_path).resize((128, 128))
                            img = np.array(img, dtype=np.float32) / 128.0
                            all_frames.append(img)
                        except IOError as e:
                            print(f"Error opening file {file_path}: {e}")  # Additional error handling

            # Load ground truth sequences
            else:
                for c in sorted(listdir(dir_path)):
                    file_path = join(dir_path, c)
                    if file_path.endswith(".bmp"):
                        print(f"Attempting to open GT file: {file_path}")  # Debug print
                        if not os.path.isfile(file_path):
                            raise FileNotFoundError(f"Expected file not found: {file_path}")
                        try:
                            img = Image.open(file_path).convert('L').resize((128, 128))
                            img = np.array(img, dtype=np.float32) / 128.0  # Normalizing GT images
                            all_gt_frames.append(img)
                        except IOError as e:
                            print(f"Error opening GT file {file_path}: {e}")  # Additional error handling

            # Add to respective lists
            if all_frames:
                for stride in range(1, 3):
                    test_clips.extend(get_clips_by_stride(stride=stride, frames_list=all_frames, sequence_size=10))
            if all_gt_frames:
                for stride in range(1, 3):
                    gt_clips.extend(get_clips_by_stride(stride=stride, frames_list=all_gt_frames, sequence_size=10))
    
    for test_clip, gt_clip in zip(test_clips, gt_clips):
        test_clips_with_gt.append((test_clip, gt_clip))
          
    return split_dataset(test_clips_with_gt)  # Split dataset into 10 parts

def create_data_loaders(dataset, batch_size, is_test=False):
    data_loaders = []
    for data_split in dataset:
        if is_test:
            # For test dataset
            test_clips, gt_clips = zip(*data_split)
            test_clips = torch.tensor(np.array(test_clips)).float()
            gt_clips = torch.tensor(np.array(gt_clips)).float()
            # Combine test_clips and gt_clips into a single tensor
            combined = torch.cat((test_clips.unsqueeze(-1), gt_clips.unsqueeze(-1)), dim=1)
            tensor_dataset = TensorDataset(combined)
        else:
            # For training dataset, keep it as 5D tensor
            data_tensor = torch.tensor(np.array(data_split)).float()
            tensor_dataset = TensorDataset(data_tensor)

        loader = DataLoader(tensor_dataset, batch_size=batch_size, shuffle=not is_test, num_workers=4)
        data_loaders.append(loader)
    return data_loaders


simple_dataset = torch.randn(10, 1, 128, 128)  # 10 random 'images'
simple_loader = DataLoader(simple_dataset, batch_size=2)

for batch in simple_loader:
    print(f"Batch type: {type(batch)}, Batch shape: {batch.shape}")
    break


# Usage
train_splits = get_training_set()
for split in train_splits:
    data_tensor = torch.tensor(np.array(split)).float()
    print(f"Data tensor shape: {data_tensor.shape}")
    break  # Just inspect the first split

train_loaders = create_data_loaders(train_splits, Config.BATCH_SIZE)
for batch in train_loaders[0]:
    print(f"Batch type: {type(batch)}, Batch content: {batch}")
    break

test_splits = get_test_set()
test_loaders = create_data_loaders(test_splits, Config.BATCH_SIZE, is_test=True)


Batch type: <class 'torch.Tensor'>, Batch shape: torch.Size([2, 1, 128, 128])
Data tensor shape: torch.Size([136, 10, 128, 128, 1])
Batch type: <class 'list'>, Batch content: [tensor([[[[[0.7969],
           [0.7812],
           [1.0000],
           ...,
           [0.7109],
           [0.4766],
           [0.6172]],

          [[0.6875],
           [0.8125],
           [1.0000],
           ...,
           [0.7578],
           [0.5156],
           [0.6719]],

          [[0.8750],
           [0.7656],
           [0.8047],
           ...,
           [0.8047],
           [0.6875],
           [0.7031]],

          ...,

          [[0.6094],
           [0.9375],
           [0.9062],
           ...,
           [1.4375],
           [1.4297],
           [1.5000]],

          [[0.6719],
           [0.9688],
           [0.9453],
           ...,
           [1.4609],
           [1.4375],
           [1.5156]],

          [[0.6953],
           [0.9922],
           [1.0000],
           ...,
         

In [18]:
class ConvLSTMCell(nn.Module):
    def __init__(self, input_size, hidden_size, kernel_size):
        super(ConvLSTMCell, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.kernel_size = kernel_size
        self.padding = kernel_size // 2
        self.conv = nn.Conv2d(input_size + hidden_size, 4 * hidden_size, kernel_size, 1, self.padding)

    def forward(self, x, state):
        h_cur, c_cur = state
        combined = torch.cat([x, h_cur], dim=1)
        gates = self.conv(combined)
        ingate, forgetgate, cellgate, outgate = gates.chunk(4, 1)
        ingate = torch.sigmoid(ingate)
        forgetgate = torch.sigmoid(forgetgate)
        cellgate = torch.tanh(cellgate)
        outgate = torch.sigmoid(outgate)

        c_next = (forgetgate * c_cur) + (ingate * cellgate)
        h_next = outgate * torch.tanh(c_next)

        return h_next, c_next
    
    def initialize_hidden_state(self, batch_size, image_size):
        height, width = image_size
        return (torch.zeros(batch_size, self.hidden_size, height, width, device=self.conv.weight.device),
                torch.zeros(batch_size, self.hidden_size, height, width, device=self.conv.weight.device))


class SimplifiedAnomalyDetectionModel(nn.Module):
    def __init__(self):
        super(SimplifiedAnomalyDetectionModel, self).__init__()
        # Encoder - simplified
        self.encoder_conv1 = nn.Conv2d(1, 64, kernel_size=5, stride=2, padding=2)
        self.encoder_conv_lstm = ConvLSTM(64, 32, kernel_size=3, num_layers=1)

        # Decoder - simplified
        self.decoder_conv_transpose = nn.ConvTranspose2d(32, 64, kernel_size=5, stride=2, padding=2, output_padding=1)
        self.decoder_conv2 = nn.Conv2d(64, 1, kernel_size=5, padding=2)

    def forward(self, x):
        # Reshape for time distributed operations
        b, t, h, w, c = x.size()
        x = x.view(b*t, c, h, w)
        
        # Encoder
        x = self.encoder_conv1(x)
        
        # Reshape for LSTM layer
        x = x.view(b, t, x.size(1), x.size(2), x.size(3))
        
        x, _ = self.encoder_conv_lstm(x)

        # Reshape for decoder conv layers
        x = x.view(b*t, x.size(2), x.size(3), x.size(4))
        
        # Decoder
        x = self.decoder_conv_transpose(x)
        x = self.decoder_conv2(x)

        x = x[:, :, :128, :128]

        # Reshape back to original shape
        x = x.view(b, t, h, w, c)
        
        return x


In [19]:
def train_model_2(model, train_loader, optimizer, criterion, device, local_epochs):
    model.train()
    for epoch in range(local_epochs):
        epoch_loss = 0.0
        for batch in train_loader:  # Now, batch is a list containing a single tensor
            sequences = batch[0].to(device)  # Extract the tensor and move it to the device

            optimizer.zero_grad()

            # Since your model seems to accept 5D tensors, ensure sequences is in that shape
            # sequences should be of shape [batch, time_steps, height, width, channels]
            outputs = model(sequences)

            # Reshape as necessary for the loss function
            b, t, h, w, c = sequences.size()
            loss = criterion(outputs.view(b * t, c, h, w), sequences.view(b * t, c, h, w))
            loss.backward()

            optimizer.step()
            epoch_loss += loss.item()

        avg_epoch_loss = epoch_loss / len(train_loader)
        print(f"Local Epoch [{epoch+1}/{local_epochs}], Loss: {avg_epoch_loss:.4f}")




In [20]:
def get_parameters(net) -> List[np.ndarray]:
    return [val.cpu().numpy() for _, val in net.state_dict().items()]


def set_parameters(net, parameters: List[np.ndarray]):
    params_dict = zip(net.state_dict().keys(), parameters)
    state_dict = OrderedDict({k: torch.Tensor(v) for k, v in params_dict})
    net.load_state_dict(state_dict, strict=True)

In [21]:
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, net, trainloader, valloader):
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader

    def get_parameters(self, config):
        return get_parameters(self.net)

    def fit(self, parameters, config):
        set_parameters(self.net, parameters)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model = SimplifiedAnomalyDetectionModel().to(device)
        optimizer = optim.Adam(model.parameters(), lr=1e-4)
        criterion = nn.MSELoss()
        train_model_2(self.net, self.trainloader, optimizer, criterion, device, local_epochs=1)
        return get_parameters(self.net), len(self.trainloader), {}
    
    
    def evaluate(self, parameters, config):
        set_parameters(self.net, parameters)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        loss, accuracy = evaluate(self.net, self.valloader, device)  # Modify this function to handle the new data format
        return float(loss), len(self.valloader), {"accuracy": float(accuracy)}

In [22]:
def client_fn(cid: str) -> FlowerClient:
    """Create a Flower client representing a single organization."""

    # Load model
    net = AnomalyDetectionModel().to(DEVICE)

    # Load data (CIFAR-10)
    # Note: each client gets a different trainloader/valloader, so each client
    # will train and evaluate on their own unique data
    trainloader = train_loaders[int(cid)]
    valloader = test_loaders[int(cid)]

    # Create a  single Flower client representing a single organization
    return FlowerClient(net, trainloader, valloader)

In [23]:
# Create FedAvg strategy
strategy = fl.server.strategy.FedAvg(
    fraction_fit=1.0,  # Sample 100% of available clients for training
    fraction_evaluate=0.5,  # Sample 50% of available clients for evaluation
    min_fit_clients=10,  # Never sample less than 10 clients for training
    min_evaluate_clients=5,  # Never sample less than 5 clients for evaluation
    min_available_clients=10,  # Wait until all 10 clients are available
)

# Specify client resources if you need GPU (defaults to 1 CPU and 0 GPU)
client_resources = None
if DEVICE.type == "cuda":
    client_resources = {"num_cpus": 1, "num_gpus": 0}

"""

# Start simulation
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=5),
    strategy=strategy,
    client_resources=client_resources,
)
"""

'\n\n# Start simulation\nfl.simulation.start_simulation(\n    client_fn=client_fn,\n    num_clients=NUM_CLIENTS,\n    config=fl.server.ServerConfig(num_rounds=5),\n    strategy=strategy,\n    client_resources=client_resources,\n)\n'

In [24]:
def weighted_average(metrics: List[Tuple[int, Metrics]]) -> Metrics:
    # Multiply accuracy of each client by number of examples used
    accuracies = [num_examples * m["accuracy"] for num_examples, m in metrics]
    examples = [num_examples for num_examples, _ in metrics]

    # Aggregate and return custom metric (weighted average)
    return {"accuracy": sum(accuracies) / sum(examples)}

In [25]:
# Create FedAvg strategy
strategy = fl.server.strategy.FedAvg(
    fraction_fit=1.0,
    fraction_evaluate=0.5,
    min_fit_clients=10,
    min_evaluate_clients=5,
    min_available_clients=10,
    evaluate_metrics_aggregation_fn=weighted_average,  # <-- pass the metric aggregation function
)

# Start simulation
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=5),
    strategy=strategy,
    client_resources=client_resources,
)

INFO flwr 2023-11-18 21:41:50,920 | app.py:175 | Starting Flower simulation, config: ServerConfig(num_rounds=5, round_timeout=None)
2023-11-18 21:41:56,285	INFO worker.py:1621 -- Started a local Ray instance.
INFO flwr 2023-11-18 21:42:03,938 | app.py:210 | Flower VCE: Ray initialized with resources: {'CPU': 16.0, 'node:__internal_head__': 1.0, 'memory': 2333021799.0, 'object_store_memory': 1166510899.0, 'node:127.0.0.1': 1.0, 'GPU': 1.0}
INFO flwr 2023-11-18 21:42:03,938 | app.py:224 | Flower VCE: Resources for each Virtual Client: {'num_cpus': 1, 'num_gpus': 0}
INFO flwr 2023-11-18 21:42:03,970 | app.py:270 | Flower VCE: Creating VirtualClientEngineActorPool with 16 actors
INFO flwr 2023-11-18 21:42:03,970 | server.py:89 | Initializing global parameters
INFO flwr 2023-11-18 21:42:03,978 | server.py:276 | Requesting initial parameters from one random client
INFO flwr 2023-11-18 21:42:47,375 | server.py:280 | Received initial parameters from one random client
INFO flwr 2023-11-18 21:42

(DefaultActor pid=13156) Local Epoch [1/1], Loss: 0.7042
(DefaultActor pid=14924) Local Epoch [1/1], Loss: 0.6836
(DefaultActor pid=12344) Local Epoch [1/1], Loss: 0.7146
(DefaultActor pid=16808) Local Epoch [1/1], Loss: 0.7231


(DefaultActor pid=4976) 
(DefaultActor pid=4976) Traceback (most recent call last):
(DefaultActor pid=4976)   File "c:\Users\akank\AppData\Local\Programs\Python\Python311\Lib\site-packages\ray\_private\serialization.py", line 387, in deserialize_objects
(DefaultActor pid=4976)     obj = self._deserialize_object(data, metadata, object_ref)
(DefaultActor pid=4976)           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
(DefaultActor pid=4976)   File "c:\Users\akank\AppData\Local\Programs\Python\Python311\Lib\site-packages\ray\_private\serialization.py", line 268, in _deserialize_object
(DefaultActor pid=4976)     return self._deserialize_msgpack_data(data, metadata_fields)
(DefaultActor pid=4976)            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
(DefaultActor pid=4976)   File "c:\Users\akank\AppData\Local\Programs\Python\Python311\Lib\site-packages\ray\_private\serialization.py", line 223, in _deserialize_msgpack_data
(DefaultActor pid=4976)     python_objects = sel